<div style="background: linear-gradient(90deg, #e6f2ff 60%, #b3c6ff 100%); padding:28px; border-radius:16px; box-shadow:0 2px 8px #00004722;">
<img src='alinco.png' width="120" style="float:left; margin-right:28px; border-radius:8px; box-shadow:0 2px 8px #00004733;"/>
<div style="margin-left:150px;">
<h1 style="color:#000047; font-size:2.3em; margin-bottom:0;">Proyecto Final de Aplicación</h1>
<h2 style="color:#003366; font-size:1.3em; margin-top:0;">Detección de Género y Edad</h2>
</div>
<br style="clear:both"/>
</div>

<div style="border-left:6px solid #000047; padding:18px; margin-top:18px; background:#f5f5f5; border-radius:8px;">
<span style="font-size:1.1em;"><b>Descripción:</b> Desarrolla un modelo de IA capaz de identificar el género (<b>Masculino</b> o <b>Femenino</b>) y estimar el rango de edad de una persona a partir de una sola imagen de rostro. El problema se aborda como clasificación en rangos de edad, debido a la dificultad de estimar una edad exacta por factores como maquillaje, iluminación y expresiones faciales.</span>
<ul style="margin-top:10px;">
<li>Rangos de edad: (0-2), (4-6), (8-12), (15-20), (25-32), (38-43), (48-53), (60-100)</li>
<li>Género: Masculino, Femenino</li>
<li>Datasets recomendados: <a href="https://fei.edu.br/~cet/facedatabase.html" target="_blank">Faces Age</a> </li>
</ul>
</div>

<div style="margin-top:18px; background:#f0f7ff; border-radius:8px; padding:14px; border:1px solid #b3c6ff;">
<b>Requerimientos:</b>
<ul style="margin-bottom:0;">
<li>Entrena una red y visualiza el progreso del <b>accuracy</b> en entrenamiento y validación.</li>
<li>Prueba distintas configuraciones de hiperparámetros para seleccionar la mejor.</li>
<li>Explica el funcionamiento y la razón de uso de la función de pérdida <b>categorical cross entropy</b>.</li>
</ul>
</div>

<div style="margin-top:18px; background:#fffbe6; border-radius:8px; padding:12px; border:1px solid #ffe066;">
<b>Condiciones de entrega:</b>
<ul>
<li>El proyecto se realiza en equipo.</li>
<li>Subir a la plataforma ALINCO un archivo <b>.zip</b> con el código, diapositivas y reporte.</li>
<li>Presentar el código y funcionamiento en una videollamada.</li>
</ul>
</div>

<div style="margin-top:18px; background:#f5f5f5; border-radius:8px; padding:12px; border:1px solid #b3c6ff;">
<b>Bibliografía recomendada:</b>
<ul>
<li><a href="https://riuma.uma.es/xmlui/bitstream/handle/10630/15226/Memoria.pdf?sequence=1&isAllowed=y" target="_blank">Trabajo de tesis 2</a></li>
</ul>
</div>

In [1]:
# -*- coding: utf-8 -*-
"""proyecto finlal.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1gKkd_0wnoM7piBjK1aeSVY9ul2ZjeS3H
"""

#adjunto mi zip con las imagnes
from google.colab import files
uploaded = files.upload()

#descomprime el zip
import os, zipfile

zip_path = list(uploaded.keys())[0]  # obtiene el nombre del archivo subido
extract_dir = './data'
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Archivos descomprimidos en:", extract_dir)

#verifico las imagenes
import os

exts = ('.jpg', '.jpeg', '.png')
total = 0
for root, dirs, files in os.walk(extract_dir):
    imgs = [f for f in files if f.lower().endswith(exts)]
    if imgs:
        print(f"{root}: {len(imgs)} imágenes")
        total += len(imgs
)
print("Total de imágenes:", total)

# Creamos un dataframe con etiquetas
import pandas as pd
import re
import os

def asignar_rango(edad):
    if edad <= 2: return 0
    elif edad <= 6: return 1
    elif edad <= 12: return 2
    elif edad <= 20: return 3
    elif edad <= 32: return 4
    elif edad <= 43: return 5
    elif edad <= 53: return 6
    else: return 7

data = []
for root, dirs, files in os.walk(extract_dir):
    for f in files:
        if f.lower().endswith(exts):
            # Modified regex to match the pattern 'age-gender.jpg'
            m = re.match(r'(\d+)-(\d+)\.jpg', f)
            if m:
                edad = int(m.group(1))
                genero = int(m.group(2))  # 0=Masculino, 1=Femenino
                data.append([os.path.join(root, f), asignar_rango(edad), genero])

df = pd.DataFrame(data, columns=['filename', 'age_class', 'gender'])
print(df.head(), df.shape)

#dividimos en entreamiento y validacion

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df[['age_class','gender']])

!pip install --upgrade tensorflow keras

#Crear datasets con tf.data

import tensorflow as tf

# Configuración
img_size = (128, 128)
batch_size = 32
num_age_classes = 8

# Función para leer y preprocesar una imagen desde la ruta
def load_and_preprocess_image(path):
    img_bytes = tf.io.read_file(path)
    img = tf.image.decode_image(img_bytes, channels=3, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)          # [0,1]
    img = tf.image.resize(img, img_size)
    return img

# Función que arma el ejemplo: (imagen, {edad_onehot, genero})
def make_example(path, age_class, gender):
    img = load_and_preprocess_image(path)
    age_onehot = tf.one_hot(age_class, depth=num_age_classes)     # one-hot
    gender = tf.cast(gender, tf.float32)                          # 0.0 / 1.0
    return img, {'age_output': age_onehot, 'gender_output': gender}

# Extraer tensores desde los DataFrames
train_paths   = tf.constant(train_df['filename'].values)
train_ages    = tf.constant(train_df['age_class'].values, dtype=tf.int32)
train_genders = tf.constant(train_df['gender'].values,    dtype=tf.int32)

val_paths   = tf.constant(val_df['filename'].values)
val_ages    = tf.constant(val_df['age_class'].values, dtype=tf.int32)
val_genders = tf.constant(val_df['gender'].values,    dtype=tf.int32)

# Crear datasets con tf.data
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_ages, train_genders))
train_ds = train_ds.map(make_example, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(2048).batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_ages, val_genders))
val_ds = val_ds.map(make_example, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

#Definir el modelo CNN y entrenar con data augmentation

import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten,
                                     Dense, Dropout, RandomFlip, RandomRotation, RandomZoom)

num_age_classes = 8

# Bloque de augmentación (solo se aplica en entrenamiento)
data_augmentation = Sequential([
    RandomFlip('horizontal'),
    RandomRotation(0.05),
    RandomZoom(0.1)
], name='augmentation')

inputs = Input(shape=(img_size[0], img_size[1], 3))
x = data_augmentation(inputs)

x = Conv2D(32, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(128, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

age_out = Dense(num_age_classes, activation='softmax', name='age_output')(x)
gender_out = Dense(1, activation='sigmoid', name='gender_output')(x)

model = Model(inputs=inputs, outputs=[age_out, gender_out])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss={'age_output': 'categorical_crossentropy', 'gender_output': 'binary_crossentropy'},
    metrics={'age_output': 'accuracy', 'gender_output': 'accuracy'}
)

model.summary()

# Entrenar
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

#Visualizar resultados de accuracy a prueba de cambios de nombres

import matplotlib.pyplot as plt

# Ver claves disponibles
print("Claves en history.history:")
print(history.history.keys())

# Helper para obtener una curva si existe
def get_curve(key):
    return history.history.get(key, [])

plt.figure(figsize=(10,5))
plt.plot(get_curve('age_output_accuracy'),       label='Edad - Train')
plt.plot(get_curve('val_age_output_accuracy'),   label='Edad - Val')
plt.plot(get_curve('gender_output_accuracy'),    label='Género - Train')
plt.plot(get_curve('val_gender_output_accuracy'),label='Género - Val')

# Si no hubo métricas por salida, intenta las generales:
if not get_curve('age_output_accuracy') and get_curve('accuracy'):
    plt.plot(get_curve('accuracy'), label='Accuracy - Train (global)')
if not get_curve('val_age_output_accuracy') and get_curve('val_accuracy'):
    plt.plot(get_curve('val_accuracy'), label='Accuracy - Val (global)')

plt.title('Progreso de Accuracy')
plt.xlabel('Épocas')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

#Explicación clara de categorical_crossentropy

print("""
¿Por qué 'categorical_crossentropy' para la edad?

• Problema multiclase: la edad se modela por rangos (8 categorías).
• La red produce una distribución de probabilidad (softmax) sobre esas 8 clases.
• La etiqueta real se representa en one-hot (ej., [0,0,1,0,0,0,0,0]).
• 'categorical_crossentropy' mide la divergencia entre la distribución real y la predicha.
• Penaliza fuertemente dar poca probabilidad a la clase correcta, forzando ajustes de pesos.
• Resultado: el modelo aprende a concentrar probabilidad donde corresponde.

Para el género (binario), 'binary_crossentropy' es la elección correcta porque solo hay
dos clases (Masculino/Femenino) y la salida es una probabilidad con 'sigmoid' (0..1).
""")

ModuleNotFoundError: No module named 'google.colab'